In [1]:
from sklearn.datasets import fetch_openml
## 需要重新加载mnist数据时候可以使用
mnist = fetch_openml('mnist_784', as_frame=False, parser='auto')
X, y = mnist.data, mnist.target
X_train, y_train, X_test, y_test = X[:60000], y[:60000], X[60000:], y[60000:]

# 1

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_train)
acc = knn_clf.score(X_test, y_test)
acc

0.9688

In [7]:
param_grid = [
    {'weights': ['uniform', 'distance'], 'n_neighbors': [2, 4, 6, 8]},
]

grid_search = GridSearchCV(knn_clf, param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train[:10000], y_train[:10000])
grid_search.best_params_

{'n_neighbors': 4, 'weights': 'distance'}

In [8]:
knn_clf = KNeighborsClassifier(weights='distance', n_neighbors=4)
knn_clf.fit(X_train, y_train)
acc = knn_clf.score(X_test, y_test)
acc

0.9714

# 2

In [11]:
from scipy.ndimage import shift

shift(X_train[0], [1, 0], cval=0)

RuntimeError: sequence argument must have length equal to input rank